<a href="https://colab.research.google.com/github/TrishankV/Music-Genre-Classifier/blob/main/Music_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!cd $PROJECT_ROOT
!mkdir -p data/GTZAN
!cd data/GTZAN

In [3]:
!if [ ! -f ~/.kaggle/kaggle.json ]; then
!    echo "Error: ~/.kaggle/kaggle.json does not exist!"
!    echo "Please create your kaggle.json file from https://www.kaggle.com/<username>/account and place it under ~/.kaggle/kaggle.json, then run this script."
!    exit 1
!fi

/bin/bash: -c: line 2: syntax error: unexpected end of file
Error: ~/.kaggle/kaggle.json does not exist!
Please create your kaggle.json file from https://www.kaggle.com/<username>/account and place it under ~/.kaggle/kaggle.json, then run this script.
/bin/bash: -c: line 1: syntax error near unexpected token `fi'
/bin/bash: -c: line 1: `fi'


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification
!unzip gtzan-dataset-music-genre-classification.zip

In [5]:
!ln -s Data/genres_original ./genres


In [6]:
!wget https://raw.githubusercontent.com/coreyker/dnn-mgr/bdad579ea6cb37b665ea6019fe1026a6ce20cbc7/gtzan/train_filtered.txt
# jazz.00054.wav is corrupted in kaggle source, remove it from train_filtered.txt
!sed -e '/jazz.00054.wav/d' train_filtered.txt > _train_filtered.txt
!mv _train_filtered.txt train_filtered.txt
!wget https://raw.githubusercontent.com/coreyker/dnn-mgr/bdad579ea6cb37b665ea6019fe1026a6ce20cbc7/gtzan/valid_filtered.txt
!wget https://raw.githubusercontent.com/coreyker/dnn-mgr/bdad579ea6cb37b665ea6019fe1026a6ce20cbc7/gtzan/test_filtered.txt

# beat tracking label
!wget http://anasynth.ircam.fr/home/system/files/attachment_uploads/marchand/private/GTZAN-Rhythm_v2_ismir2015_lbd_2015-10-28.tar_.gz


--2024-07-28 17:00:34--  https://raw.githubusercontent.com/coreyker/dnn-mgr/bdad579ea6cb37b665ea6019fe1026a6ce20cbc7/gtzan/train_filtered.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10152 (9.9K) [text/plain]
Saving to: ‘train_filtered.txt’

train_filtered.txt  100%[===================>]   9.91K  --.-KB/s    in 0s      

2024-07-28 17:00:34 (94.0 MB/s) - ‘train_filtered.txt’ saved [10152/10152]

--2024-07-28 17:00:34--  https://raw.githubusercontent.com/coreyker/dnn-mgr/bdad579ea6cb37b665ea6019fe1026a6ce20cbc7/gtzan/valid_filtered.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [ ]:
!tar xzvf GTZAN-Rhythm_v2_ismir2015_lbd_2015-10-28.tar_.gz

!cd $PROJECT_ROOT

In [ ]:
# prompt: list all the files and every file in DATA
!ls -l /content/Data



total 11908
-rw-r--r--  1 root root  1108271 Mar 24  2020 features_30_sec.csv
-rw-r--r--  1 root root 11075018 Mar 24  2020 features_3_sec.csv
drwxr-xr-x 12 root root     4096 Jul 28 13:46 genres_original
drwxr-xr-x 12 root root     4096 Jul 28 13:46 images_original


In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import soundfile as sf
import audioread


In [2]:
def extract_features(file_name):
  try :

    audio , sample_rate = librosa.load(file_name , res_type = "kaiser_fast")
    mel = librosa.feature.melspectrogram(y = audio , sr = sample_rate)
    mel_db = librosa.power_to_db(mel)
    return mel_db
  except (sf.LibsndfileError , audioread.NoBackendError) as e :
    print(f"Error: {file_name , e}")
    return None

In [3]:
path = '/content/Data/genres_original'

In [4]:
gen_name = os.listdir(path)
gen_name.sort()

In [5]:
gen_name

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [6]:
Dict = [os.path.join(path , i ) for i in gen_name]
print(Dict)

['/content/Data/genres_original/blues', '/content/Data/genres_original/classical', '/content/Data/genres_original/country', '/content/Data/genres_original/disco', '/content/Data/genres_original/hiphop', '/content/Data/genres_original/jazz', '/content/Data/genres_original/metal', '/content/Data/genres_original/pop', '/content/Data/genres_original/reggae', '/content/Data/genres_original/rock']


In [7]:
Dictionary =  {name: index for index, name in enumerate(gen_name)}


In [8]:
Dictionary

{'blues': 0,
 'classical': 1,
 'country': 2,
 'disco': 3,
 'hiphop': 4,
 'jazz': 5,
 'metal': 6,
 'pop': 7,
 'reggae': 8,
 'rock': 9}

In [9]:
data , labels = [] , []

In [10]:
for i in Dict:
  for j in os.listdir(i):
    if j.endswith('.wav'):
      mel_spectogram /= extract_features(os.path.join(i , j))
      if mel_spectogram is not None:
        data.append(mel_spectogram)
        labels.append(Dictionary[i.split('/')[-1]])


<ipython-input-2-a65556fedc4b>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio , sample_rate = librosa.load(file_name , res_type = "kaiser_fast")
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error: ('/content/Data/genres_original/jazz/jazz.00054.wav', NoBackendError())


In [11]:
len(data)

999

In [12]:
len(labels)

999

In [13]:
import cv2

In [14]:
def resize_data(data):
  return cv2.resize(data , (128 , 128))


In [15]:
r_data = [resize_data(i) for i in data if i is not None]
r_data = np.array(r_data)

In [16]:
r_data.shape

(999, 128, 128)

In [17]:
labels = labels[:len(r_data)]
labels = np.array(labels)

In [18]:
labels.shape

(999,)

In [19]:
r_data = r_data.reshape(r_data.shape[0] , 128 , 128 , 1)

In [20]:
r_data = r_data / 255.0

In [21]:
from sklearn.preprocessing import LabelEncoder as le
from tensorflow.keras.utils import to_categorical

In [22]:
le = le()
labels = le.fit_transform(labels)
labels = to_categorical(labels)

In [23]:
labels.shape

(999, 10)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train , X_test , y_train , y_test = train_test_split(r_data , labels , test_size = 0.2 , random_state = 42)

In [26]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(799, 128, 128, 1)
(200, 128, 128, 1)
(799, 10)
(200, 10)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer , Conv2D , MaxPooling2D , Flatten , Dense , Dropout , BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [34]:
r_s = 0.0001

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape = (128 , 128 , 1)))
model.add(InputLayer(input_shape=(128, 128, 1)))
model.add(Conv2D(32, (5, 5), activation='relu' , kernel_regularizer=l2(r_s)))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(Conv2D(64, (5, 5), activation='relu' , kernel_regularizer=l2(r_s)))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(Conv2D(128, (5, 5), activation='relu' , kernel_regularizer=l2(r_s)))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_regularizer=l2(r_s)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu',kernel_regularizer=l2(r_s)))
model.add(Dense(10, activation='softmax',kernel_regularizer=l2(r_s)))  # Output layer with softmax for classification
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        multiple                  0         
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 32)      832       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 58, 58, 64)        51264     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 29, 29, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 25, 25, 128)      

In [ ]:
model.fit(X_train , y_train , epochs = 50 , batch_size = 32 , validation_data = (X_test , y_test))

Epoch 1/50
25/25 [==============================] - 49s 2s/step - loss: 2.2681 - accuracy: 0.1790 - val_loss: 2.2488 - val_accuracy: 0.1600
Epoch 2/50
25/25 [==============================] - 45s 2s/step - loss: 2.1038 - accuracy: 0.1902 - val_loss: 2.0286 - val_accuracy: 0.2000
Epoch 3/50
25/25 [==============================] - 45s 2s/step - loss: 2.0311 - accuracy: 0.2516 - val_loss: 2.0110 - val_accuracy: 0.2700
Epoch 4/50
25/25 [==============================] - 45s 2s/step - loss: 1.9478 - accuracy: 0.2979 - val_loss: 1.9283 - val_accuracy: 0.2750
Epoch 5/50
25/25 [==============================] - 47s 2s/step - loss: 1.8826 - accuracy: 0.3317 - val_loss: 1.7629 - val_accuracy: 0.3750
Epoch 6/50
25/25 [==============================] - 45s 2s/step - loss: 1.8445 - accuracy: 0.3479 - val_loss: 1.7558 - val_accuracy: 0.3400
Epoch 7/50
25/25 [==============================] - 45s 2s/step - loss: 1.6853 - accuracy: 0.3842 - val_loss: 1.6201 - val_accuracy: 0.4050
Epoch 8/50
25/25 [==

In [ ]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred , axis = 1)
y_test2 = np.argmax(y_test , axis = 1)

7/7 [==============================] - 3s 343ms/step


In [ ]:
print(accuracy_score(y_test2 , y_pred)*100 ,'%')
print(confusion_matrix(y_test2 , y_pred))
print(classification_report(y_test2 , y_pred))

68.0 %
[[16  0  1  2  0  0  0  0  1  1]
 [ 0 12  0  0  0  0  0  0  0  0]
 [ 1  0 12  5  0  2  0  1  1  2]
 [ 0  0  0 17  1  0  1  2  0  1]
 [ 0  0  0  0 12  0  0  1  1  1]
 [ 3  2  1  0  0 19  0  0  2  0]
 [ 0  0  0  2  1  0 14  0  0  1]
 [ 0  0  1  1  0  0  0 16  1  0]
 [ 0  0  1  1  1  0  0  1 15  3]
 [ 2  0  0  8  2  0  3  2  0  3]]
              precision    recall  f1-score   support

           0       0.73      0.76      0.74        21
           1       0.86      1.00      0.92        12
           2       0.75      0.50      0.60        24
           3       0.47      0.77      0.59        22
           4       0.71      0.80      0.75        15
           5       0.90      0.70      0.79        27
           6       0.78      0.78      0.78        18
           7       0.70      0.84      0.76        19
           8       0.71      0.68      0.70        22
           9       0.25      0.15      0.19        20

    accuracy                           0.68       200
   macro avg

In [37]:
!pip install audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xgboost 2.1.0 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
osqp 0.6.7.post0 requires scipy!=1.12.0,>=0.13.2, but you have scipy 1.12.0 which is incompatible.


In [38]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

In [39]:
augment = Compose([
    AddGaussianNoise(min_amplitude = 0.001 ,
                     max_amplitude = 0.015 ,
                     p = 0.5) ,
    TimeStretch(min_rate = 0.8 ,
                max_rate = 1.25 ,
                p = 0.5 ),
    PitchShift(min_semitones = -4 ,
               max_semitones = 4 ,
               p = 0.5)
])

In [40]:
def augment_audio(y):
    # Apply augmentation
    y_augmented = augment(samples=y, sample_rate=22050)
    return y_augmented

In [41]:
def extract_features2(file_name, fixed_length=128):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        # Apply augmentation
        audio = augment_audio(audio)
        # Extract Mel spectrogram features
        mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
        mel_db = librosa.power_to_db(mel)

        # Pad or truncate to fixed length
        if mel_db.shape[1] > fixed_length:
            mel_db = mel_db[:, :fixed_length]
        else:
            padding = fixed_length - mel_db.shape[1]
            mel_db = np.pad(mel_db, ((0, 0), (0, padding)), mode='constant')

        return mel_db
    except (sf.LibsndfileError, audioread.NoBackendError) as e:
        print(f"Error: {file_name} - {e}")
        return None

In [42]:
x_data = []
y_data = []

for class_idx, class_name in enumerate(gen_name):
    class_dir = os.path.join(path, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        features = extract_features2(file_path)
        if features is not None:
            x_data.append(features)
            y_data.append(class_idx)



<ipython-input-41-6bf8d1100b43>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error: /content/Data/genres_original/jazz/jazz.00054.wav - 


In [43]:
x_data = np.array(x_data)
y_data = to_categorical(np.array(y_data), num_classes=len(gen_name))

In [ ]:
x_train, x_val, Y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=42)


In [45]:
x_train.shape

(799, 128, 128)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=((128, 128 , 1)), kernel_regularizer=l2(r_s)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(r_s)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(r_s)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(r_s)))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(r_s)))
model.add(Dense(10, activation='softmax', kernel_regularizer=l2(r_s)))

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 126, 126, 32)      320       
                                                                 
 batch_normalization_7 (Bat  (None, 126, 126, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_8 (Bat  (None, 61, 61, 64)        256       
 chNormalization)                                                
                                                      

In [ ]:
history = model.fit(x_train, Y_train, epochs=50, validation_data=(x_val, y_val))


Epoch 1/50
25/25 [==============================] - 34s 1s/step - loss: 2.7183 - accuracy: 0.1502 - val_loss: 2.3879 - val_accuracy: 0.1450
Epoch 2/50
25/25 [==============================] - 31s 1s/step - loss: 2.1233 - accuracy: 0.2591 - val_loss: 2.3307 - val_accuracy: 0.1150
Epoch 3/50
25/25 [==============================] - 29s 1s/step - loss: 1.9094 - accuracy: 0.3304 - val_loss: 2.3209 - val_accuracy: 0.1200
Epoch 4/50
25/25 [==============================] - 31s 1s/step - loss: 1.7181 - accuracy: 0.4143 - val_loss: 2.2942 - val_accuracy: 0.1400
Epoch 5/50
25/25 [==============================] - 31s 1s/step - loss: 1.6327 - accuracy: 0.4155 - val_loss: 2.1989 - val_accuracy: 0.1950
Epoch 6/50
25/25 [==============================] - 31s 1s/step - loss: 1.3983 - accuracy: 0.5156 - val_loss: 2.1163 - val_accuracy: 0.2450
Epoch 7/50
25/25 [==============================] - 31s 1s/step - loss: 1.2411 - accuracy: 0.6083 - val_loss: 1.9582 - val_accuracy: 0.3200
Epoch 8/50
25/25 [==

In [ ]:
Y_train.shape

(799, 10)

In [ ]:
# prompt: print accutrcat confession matrix

y_pred = model.predict(x_val)
y_pred = np.argmax(y_pred, axis=1)
y_test2 = np.argmax(y_val, axis=1)
print(accuracy_score(y_test2, y_pred) * 100, '%')
print(confusion_matrix(y_test2, y_pred))
print(classification_report(y_test2, y_pred))


7/7 [==============================] - 2s 215ms/step
38.5 %
[[ 5  4  1  1  1  4  1  1  1  2]
 [ 0 11  0  1  0  0  0  0  0  0]
 [ 1  9  2  5  0  2  1  0  0  4]
 [ 1  0  1  6  1  0  4  3  1  5]
 [ 0  0  0  1  9  0  3  0  1  1]
 [ 2 10  0  0  1 11  0  0  2  1]
 [ 0  0  0  0  1  0 17  0  0  0]
 [ 0  1  1  5  4  1  0  4  2  1]
 [ 0  0  0  4  7  0  0  0  8  3]
 [ 2  3  1  4  1  0  4  0  1  4]]
              precision    recall  f1-score   support

           0       0.45      0.24      0.31        21
           1       0.29      0.92      0.44        12
           2       0.33      0.08      0.13        24
           3       0.22      0.27      0.24        22
           4       0.36      0.60      0.45        15
           5       0.61      0.41      0.49        27
           6       0.57      0.94      0.71        18
           7       0.50      0.21      0.30        19
           8       0.50      0.36      0.42        22
           9       0.19      0.20      0.20        20

    accuracy 

LEts try a RNN

In [47]:
from keras.layers import LSTM , Reshape , Bidirectional

In [55]:
RNN = Sequential()
RNN.add(InputLayer(input_shape=(128, 128, 1)))
RNN.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.0001)))
RNN.add(BatchNormalization())
RNN.add(MaxPooling2D((2, 2)))
RNN.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.0001)))
RNN.add(BatchNormalization())
RNN.add(MaxPooling2D((2, 2)))
RNN.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.0001)))
RNN.add(BatchNormalization())
RNN.add(MaxPooling2D((2, 2)))
# # RNN.add(Flatten())
# RNN.add(Reshape((14 * 14, 128)))
# RNN.add(LSTM(128, return_sequences=True))
# RNN.add(LSTM(128))
RNN.add(Flatten())
RNN.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0001)))
RNN.add(Dropout(0.5))
RNN.add(Dense(128, activation='relu', kernel_regularizer=l2(0.0001)))
RNN.add(Dropout(0.5))
RNN.add(Dense(64, activation='relu', kernel_regularizer=l2(0.0001)))
RNN.add(Dense(10, activation='softmax', kernel_regularizer=l2(0.0001)))

# Compile the RNN
optimizer = Adam(learning_rate=0.0001)
RNN.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



In [56]:
RNN.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 batch_normalization_10 (Ba  (None, 126, 126, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_11 (Ba  (None, 61, 61, 64)        256       
 tchNormalization)                                               
                                                      

In [59]:
RNN.fit(X_train , y_train , epochs = 50 , batch_size = 32 , validation_data = (X_test , y_test))

Epoch 1/50
25/25 [==============================] - 67s 3s/step - loss: 1.4155 - accuracy: 0.4944 - val_loss: 2.6398 - val_accuracy: 0.1150
Epoch 2/50
25/25 [==============================] - 59s 2s/step - loss: 1.4072 - accuracy: 0.4969 - val_loss: 2.5198 - val_accuracy: 0.1200
Epoch 3/50
25/25 [==============================] - 59s 2s/step - loss: 1.3982 - accuracy: 0.5031 - val_loss: 2.4338 - val_accuracy: 0.1400
Epoch 4/50
25/25 [==============================] - 59s 2s/step - loss: 1.4256 - accuracy: 0.5131 - val_loss: 2.4370 - val_accuracy: 0.1450
Epoch 5/50
25/25 [==============================] - 58s 2s/step - loss: 1.3771 - accuracy: 0.5319 - val_loss: 2.3549 - val_accuracy: 0.1850
Epoch 6/50
25/25 [==============================] - 62s 3s/step - loss: 1.3398 - accuracy: 0.5244 - val_loss: 2.4821 - val_accuracy: 0.1350
Epoch 7/50
25/25 [==============================] - 56s 2s/step - loss: 1.2906 - accuracy: 0.5544 - val_loss: 2.4079 - val_accuracy: 0.1500
Epoch 8/50
25/25 [==

In [60]:
# prompt: print the accuracy precision and confusion matrix

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# ... (your existing code) ...

# Predict on the test set
y_pred = RNN.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", conf_matrix)


7/7 [==============================] - 13s 1s/step
Accuracy: 0.575
Precision: 0.5605185015147929
Recall: 0.575
F1-score: 0.5562677460719151
Confusion Matrix:
 [[15  0  3  1  1  1  0  0  0  0]
 [ 0 11  0  0  0  1  0  0  0  0]
 [ 1  1  8  3  1  3  0  1  5  1]
 [ 0  0  0  6  8  0  0  4  2  2]
 [ 0  0  1  2  7  0  0  3  2  0]
 [ 0  3  1  0  0 22  0  1  0  0]
 [ 0  0  0  0  0  0 18  0  0  0]
 [ 0  0  3  1  2  0  0 13  0  0]
 [ 1  0  3  1  3  0  0  0 13  1]
 [ 1  0  0  6  3  2  5  0  1  2]]


In [61]:
print("Hello")

Hello


In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Add

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def create_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(r_s))(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(r_s))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(r_s))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Flatten the conv output
    x = Flatten()(x)

    # Reshape for Transformer
    x = Dense(256, activation='relu')(x)
    x = tf.expand_dims(x, axis=1)

    # Transformer block
    transformer_block = TransformerBlock(embed_dim=256, num_heads=8, ff_dim=512)
    x = transformer_block(x)
    x = tf.squeeze(x, axis=1)

    # Dense layers
    x = Dense(128, activation='relu', kernel_regularizer=l2(r_s))(x)
    x = Dropout(0.4)(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(r_s))(x)
    outputs = Dense(num_classes, activation='softmax', kernel_regularizer=l2(r_s))(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Parameters
input_shape = (128, 128, 1)
num_classes = 10
r_s = 0.01

# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 batch_normalization (Batch  (None, 126, 126, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 64)        256   

In [29]:
model.fit(X_train , y_train , epochs = 50 , batch_size = 32 , validation_data = (X_test , y_test))

Epoch 1/50
25/25 [==============================] - 53s 2s/step - loss: 6.2128 - accuracy: 0.2178 - val_loss: 6.6306 - val_accuracy: 0.1050
Epoch 2/50
25/25 [==============================] - 41s 2s/step - loss: 5.8093 - accuracy: 0.3567 - val_loss: 6.6768 - val_accuracy: 0.1000
Epoch 3/50
25/25 [==============================] - 39s 2s/step - loss: 5.5423 - accuracy: 0.4443 - val_loss: 6.8330 - val_accuracy: 0.0900
Epoch 4/50
25/25 [==============================] - 40s 2s/step - loss: 5.1368 - accuracy: 0.5732 - val_loss: 6.7965 - val_accuracy: 0.0900
Epoch 5/50
25/25 [==============================] - 43s 2s/step - loss: 4.7478 - accuracy: 0.7059 - val_loss: 6.7181 - val_accuracy: 0.0900
Epoch 6/50
25/25 [==============================] - 39s 2s/step - loss: 4.4923 - accuracy: 0.7735 - val_loss: 6.6788 - val_accuracy: 0.0900
Epoch 7/50
25/25 [==============================] - 43s 2s/step - loss: 4.1272 - accuracy: 0.9111 - val_loss: 6.6778 - val_accuracy: 0.0900
Epoch 8/50
25/25 [==

In [31]:
# prompt: print the accuracy , precision % and confusion matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# ... (your existing code) ...

# Predict on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Print the results
print("Accuracy:", accuracy * 100, "%")
print("Precision:", precision * 100, "%")
print("Recall:", recall * 100, "%")
print("F1-score:", f1 * 100, "%")
print("Confusion Matrix:\n", conf_matrix)


7/7 [==============================] - 4s 500ms/step
Accuracy: 68.0 %
Precision: 68.68764053579271 %
Recall: 68.0 %
F1-score: 67.37691464967023 %
Confusion Matrix:
 [[14  0  2  0  0  3  0  0  2  0]
 [ 0 11  0  0  0  0  0  0  0  1]
 [ 2  0 17  1  0  0  0  3  0  1]
 [ 1  0  0 14  4  0  0  2  0  1]
 [ 0  0  0  0 11  0  3  0  1  0]
 [ 1  3  2  0  0 20  0  0  1  0]
 [ 0  0  0  0  1  0 16  0  0  1]
 [ 0  1  0  1  2  0  0 14  0  1]
 [ 1  0  1  2  4  0  0  0 14  0]
 [ 1  0  2  7  2  0  3  0  0  5]]


In [33]:
# prompt: save the model to '/content'

# ... (your existing code) ...

# Save the model
model.save('/content/music_genre_transformer.h5')


In [37]:
import tensorflow_hub as hub
import tensorflow_io as tfio
yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

In [46]:
def load_wav(filename):
  auido , sample_rate = librosa.load(filename , sr = 16000 , mono = True)
  # auido = tf.cast(auido , tf.float32)
  return auido

In [57]:
def extract_features3(wav):
  waveform = tf.convert_to_tensor(wav , dtype=tf.float32)
  waveform = tf.reshape(waveform , [-1])
  scores , embedding  , spectrogram = yamnet(waveform)
  return embedding.numpy().flatten()

In [63]:
# Check the shape of each element in 'nd'
for i, element in enumerate(nd):
    print(f"Element {i} shape: {np.shape(element)}")

# If the shapes vary, consider padding or truncating the arrays
# to make them uniform before converting to a NumPy array.

Element 0 shape: (63488,)
Element 1 shape: (63488,)
Element 2 shape: (63488,)
Element 3 shape: (63488,)
Element 4 shape: (63488,)
Element 5 shape: (63488,)
Element 6 shape: (63488,)
Element 7 shape: (63488,)
Element 8 shape: (63488,)
Element 9 shape: (63488,)
Element 10 shape: (63488,)
Element 11 shape: (63488,)
Element 12 shape: (63488,)
Element 13 shape: (63488,)
Element 14 shape: (63488,)
Element 15 shape: (63488,)
Element 16 shape: (63488,)
Element 17 shape: (63488,)
Element 18 shape: (63488,)
Element 19 shape: (63488,)
Element 20 shape: (63488,)
Element 21 shape: (63488,)
Element 22 shape: (63488,)
Element 23 shape: (63488,)
Element 24 shape: (63488,)
Element 25 shape: (63488,)
Element 26 shape: (63488,)
Element 27 shape: (63488,)
Element 28 shape: (63488,)
Element 29 shape: (63488,)
Element 30 shape: (63488,)
Element 31 shape: (63488,)
Element 32 shape: (63488,)
Element 33 shape: (63488,)
Element 34 shape: (63488,)
Element 35 shape: (63488,)
Element 36 shape: (63488,)
Element 37 

In [64]:
import numpy as np

def pad_sequences(sequences, maxlen=None, dtype='int32', padding='post', truncating='post', value=0.0):
    """Pads sequences to the same length.

    Args:
        sequences: List of sequences (NumPy arrays).
        maxlen: Maximum length of all sequences. If None, set to the length of the longest sequence.
        dtype: Data type of the padded sequences.
        padding: 'pre' or 'post', pad either before or after each sequence.
        truncating: 'pre' or 'post', remove values from sequences larger than `maxlen` either
            at the beginning or the end of the sequences.
        value: Value to pad the sequences with.

    Returns:
        NumPy array with the padded sequences.
    """
    lengths = [len(s) for s in sequences]
    if maxlen is None:
        maxlen = np.max(lengths)

    num_samples = len(sequences)
    padded_sequences = np.full((num_samples, maxlen), value, dtype=dtype)
    for i, s in enumerate(sequences):
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        if padding == 'post':
            padded_sequences[i, :len(trunc)] = trunc
        elif padding == 'pre':
            padded_sequences[i, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)

    return padded_sequences

# ... (rest of your code)

# After extracting features:
nd_padded = pad_sequences(nd, dtype='float32')

# Now you can try creating the NumPy array:
nd_array = np.array(nd_padded)

In [58]:
nd , ns = [] , []

In [59]:
for genre_dir in Dict:
  genre_label = genre_dir.split('/')[-1]
  if genre_label not in Dictionary:
    continue
  label = Dictionary[genre_label]
  for file_name in os.listdir (genre_dir):
    if file_name.endswith('.wav'):
      file_path = os.path.join(genre_dir, file_name)
      wav_data = load_wav(file_path)
      if wav_data is not None:
        features = extract_features3(wav_data)
        if features is not None:
          nd.append(features)
          ns.append(label)


In [65]:
ns = np.array(ns)

In [68]:
nd_array.shape

(999, 64512)

In [72]:
nd_array = nd_array.reshape(nd_array.shape[0] , -1)

In [99]:
from tensorflow.keras.layers import LSTM , Reshape , Bidirectional

In [105]:
dif_shpe = (nd_array.shape[1] ,)
new_model = Sequential()
new_model.add(InputLayer(input_shape=dif_shpe))
new_model.add(Dense(256 , activation = 'relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(128 , activation = 'relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(64 , activation = 'relu'))
new_model.add(Dense(10 , activation = 'softmax'))
new_model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

In [106]:
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 256)               16515328  
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 dense_28 (Dense)            (None, 128)               32896     
                                                                 
 dropout_16 (Dropout)        (None, 128)               0         
                                                                 
 dense_29 (Dense)            (None, 64)                8256      
                                                                 
 dense_30 (Dense)            (None, 10)                650       
                                                                 
Total params: 16557130 (63.16 MB)
Trainable params: 1

In [107]:
new_model.fit(nd_array , ns , epochs = 50 , batch_size = 32 , validation_split = 0.2 ,
              callbacks = [
                  tf.keras.callbacks.EarlyStopping(
                      monitor='val_loss',
                      patience=3,
                      restore_best_weights=True) ,
                    tf.keras.callbacks.ReduceLROnPlateau(
                      monitor='val_loss',
                      factor=0.2,
                      patience=2,
                      min_lr=0.001)

          ])

Epoch 1/50
25/25 [==============================] - 12s 451ms/step - loss: 2.1774 - accuracy: 0.3855 - val_loss: 4.0587 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/50
25/25 [==============================] - 10s 388ms/step - loss: 1.0713 - accuracy: 0.6483 - val_loss: 5.8449 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/50
25/25 [==============================] - 10s 401ms/step - loss: 0.7678 - accuracy: 0.7422 - val_loss: 5.0721 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/50
25/25 [==============================] - 9s 375ms/step - loss: 0.7232 - accuracy: 0.7885 - val_loss: 5.6295 - val_accuracy: 0.0000e+00 - lr: 0.0010


In [108]:
# prompt: check the accuracy

# ... (your existing code) ...

# Evaluate the model
loss, accuracy = new_model.evaluate(nd_array, ns, verbose=0)
print('Accuracy: %.2f%%' % (accuracy * 100))


Accuracy: 60.66%
